# Load Dataset & Imports

In [ ]:
import pandas as pd
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error



df = pd.read_csv('yield.csv')
df.drop(columns=['Unnamed: 0'],axis=1,inplace=True)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'yield_df.csv'

# Train Test Splits

In [ ]:
# Define features and target
X = df.drop('hg/ha_yield', axis=1)
y = df['hg/ha_yield']

# Split data (optional, if you're preparing for modeling)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Pipeline

In [ ]:
# Define categorical and numerical columns
categorical_cols = ['Area', 'Item']
numerical_cols = ['Year', 'average_rain_fall_mm_per_year', 'pesticides_tonnes', 'avg_temp']

# Preprocessing for numerical data
numerical_transformer = StandardScaler()

# Preprocessing for categorical data
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Combine preprocessing for both types
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

# Full pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor())
])



pipeline.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Year',
                                                   'average_rain_fall_mm_per_year',
                                                   'pesticides_tonnes',
                                                   'avg_temp']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Area', 'Item'])])),
                ('regressor', RandomForestRegressor())])

# Evaluation

In [ ]:
y_pred = pipeline.predict(X_test)

print("score :",r2_score(y_test, y_pred))
print("mse :", mean_squared_error(y_test, y_pred))

score : 0.9878634715682127
mse : 88034396.10926507


# Prediction

In [ ]:
def predict_crop_yield(pipeline, area, item, year, average_rain_fall_mm_per_year, pesticides_tonnes, avg_temp):
    # Create a DataFrame for a single prediction
    input_data = pd.DataFrame({
        'Area': [area],
        'Item': [item],
        'Year': [year],
        'average_rain_fall_mm_per_year': [average_rain_fall_mm_per_year],
        'pesticides_tonnes': [pesticides_tonnes],
        'avg_temp': [avg_temp]
    })

    # Use the pipeline to predict
    prediction = pipeline.predict(input_data)

    return prediction[0]



# Example inputs
area = 'Albania'  # in square kilometers
item = 'Wheat'  # type of crop
year = 2025  # the year for the prediction
average_rain_fall_mm_per_year = 800  # in mm
pesticides_tonnes = 15  # amount of pesticides in tonnes
avg_temp = 22  # average temperature in Celsius

# Call the prediction function
predicted_yield = predict_crop_yield(pipeline, area, item, year, average_rain_fall_mm_per_year, pesticides_tonnes, avg_temp)

# Print the result
print(f"The predicted crop yield is: {predicted_yield} tonnes")


The predicted crop yield is: 25045.19 tonnes


In [ ]:
# save pipeline
import pickle 
pickle.dump(pipeline,open("pipeline.pkl",'wb'))